## GAN from Scratch

### Import Libraries

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as T # Transformation functions to manipulate images

import torch.nn.functional as F # various activation functions for model
import torchvision # You can load various Pretrained Model from this package 
# import torchvision.datasets as vision_dsets
import torch.optim as optim # various optimization functions for model
from torch.autograd import Variable 
# from torch.utils import data

In [2]:
import os
import numpy as np

### Visualization Setup

In [3]:
import random
import torch.backends.cudnn as cudnn
random_seed = 999
print(f"Randon Seed: {random_seed}")
random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
cudnn.benchmark = False
cudnn.deterministic = True
np.random.seed(random_seed) # for numpy-based backend, scikit-learn

Randon Seed: 999


### Dataset Preprocessing

In [4]:
MNIST_dir = "../MNIST/"
train_imgs_dir = os.path.join(MNIST_dir, 'train-images-idx3-ubyte')
train_labels_dir = os.path.join(MNIST_dir, 'train-labels-idx1-ubyte')
test_imgs_dir = os.path.join(MNIST_dir, 't10k-images-idx3-ubyte')
test_labels_dir = os.path.join(MNIST_dir, 't10k-labels-idx1-ubyte')

In [ ]:
'''
import struct
import matplotlib.pyplot as plt
import matplotlib.cm as cm

train_imgs_file = open(train_imgs_dir,'rb')
train_labels_file = open(train_labels_dir,'rb')
train_img = np.zeros((28,28))
train_label = -1
img = train_imgs_file.read(16)
label = train_labels_file.read(8)

img = train_imgs_file.read(28*28)
img = np.reshape(struct.unpack(len(img)*'B',img), (28,28))
label = train_labels_file.read(1)

plt.imshow(img, cmap='gray') 
plt.show()
'''

### Custom Dataset

In [ ]:
# import os
import pandas as pd
import struct

class MNISTDataset(Dataset):
    def __init__(self, imgs_dir, labels_dir, len, transform=None, target_transform=None):
        # open file
        self.imgs_file = open(imgs_dir,'rb')
        self.labels_file = open(labels_dir,'rb')
        # initialize
        self.image = np.zeros((28,28))
        self.label = -1
        img = self.imgs_file.read(16)
        label = self.labels_file.read(8)
        self.len = len
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # return length
        # usually directly calculated from csv idx length
        return self.len

    def __getitem__(self, idx):
        # convert file format (be careful!)
        i = self.imgs_file.read(28*28)
        i = np.reshape(struct.unpack(len(i)*'B',i), (28,28))
        img = i.astype(np.uint8)
        label = int.from_bytes(self.labels_file.read(1),'big')
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label

In [49]:
import pandas as pd
import struct

class MNISTDataset(Dataset):
    def __init__(self, imgs_dir, labels_dir, len, transform=None, target_transform=None):
        # open file
        self.imgs_file = open(imgs_dir,'rb')
        self.labels_file = open(labels_dir,'rb')
        # initialize
        dummy_imgs = self.imgs_file.read(16)
        dummy_labels = self.labels_file.read(8)
        imgs = self.imgs_file.read()
        labels = self.labels_file.read()
        # self.imgs = np.reshape(struct.unpack(len(imgs)*'B',imgs), (-1,28,28))
        self.imgs = np.frombuffer(imgs, dtype=np.uint8).reshape((-1,28,28))
        self.labels = np.frombuffer(labels,dtype=np.uint8)
        self.len = len
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # return length
        # usually directly calculated from csv idx length
        return self.len

    def __getitem__(self, idx):
        # convert file format (be careful!)
        img = self.imgs[idx].astype(np.uint8)
        label = int(self.labels[idx])
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label

### Transform

In [50]:
# PIL image or numpy array(uint8) -> float tensor ; change intensity in range [0.0, 1.1] (include normalizing)
# For data aug and transforming, we can include many transform functions using nn.Sequential
# Normalization is depend on channel (RGB)
transform = T.ToTensor()
# one-hot-vector ; make zero tensor and assign 1 on the index given by the label y
target_transform = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y),value=1))

training_data = MNISTDataset(train_imgs_dir, train_labels_dir, 60000, transform, target_transform)
test_data = MNISTDataset(test_imgs_dir, test_labels_dir, 10000, transform, target_transform)

In [51]:
print(training_data.imgs.shape)
print(training_data.labels.shape)
type(training_data.labels[0])

(60000, 28, 28)
(60000,)


numpy.uint8

### Iterating and Visulaizing the Dataset

In [23]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    # sample_idx = torch.randint(len(training_data), size=(1,)).item()
    sample_idx = torch.tensor([i+100])
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

NameError: name 'plt' is not defined

In [52]:
img, label = training_data[5990]
print(label)

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])


### Dataloader

In [53]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False) # don't need to shuffle ; model.eval()

In [ ]:
'''
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
# print(img)
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")
'''

### Build the Network

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [55]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [56]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [57]:
'''
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")
'''

'\nX = torch.rand(1, 28, 28, device=device)\nlogits = model(X)\npred_probab = nn.Softmax(dim=1)(logits)\ny_pred = pred_probab.argmax(1)\nprint(f"Predicted class: {y_pred}")\n'

In [58]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0305,  0.0011,  0.0150,  ...,  0.0066, -0.0111, -0.0283],
        [-0.0222,  0.0069,  0.0205,  ..., -0.0180, -0.0069, -0.0185]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0028, 0.0324], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0146, -0.0362,  0.0071,  ..., -0.0154, -0.0428,  0.0050],
        [ 0.0438, -0.0422,  0.0023,  ..., -0.0004,  0.0371, -0.0198]],
       device='cuda:0', grad_fn=<Slic

## Optimizing and Model Parameters

### Hyperparameters

In [59]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

### Optimization Loop

In [79]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        pred = model(X).to(device)
        y = y.to(device)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(f"batch: {batch}, x: {X.size()}, y: {y.size()}")
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X = X.to(device)
            pred = model(X).to(device)
            y = y.to(device)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.max(1).indices).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [80]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    # model.train()
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # model.eval()
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.865969  [    0/60000]
loss: 1.895601  [ 3200/60000]
loss: 1.846244  [ 6400/60000]
loss: 1.827845  [ 9600/60000]
loss: 1.714645  [12800/60000]
loss: 1.682227  [16000/60000]
loss: 1.652585  [19200/60000]
loss: 1.612659  [22400/60000]
loss: 1.740344  [25600/60000]
loss: 1.600793  [28800/60000]
loss: 1.614352  [32000/60000]
loss: 1.628732  [35200/60000]
loss: 1.545228  [38400/60000]
loss: 1.401749  [41600/60000]
loss: 1.501090  [44800/60000]
loss: 1.434402  [48000/60000]
loss: 1.202066  [51200/60000]
loss: 1.147357  [54400/60000]
loss: 1.107255  [57600/60000]
Test Error: 
 Accuracy: 76.1%, Avg loss: 1.234715 

Epoch 2
-------------------------------
loss: 1.270793  [    0/60000]
loss: 1.262928  [ 3200/60000]
loss: 1.119214  [ 6400/60000]
loss: 1.355643  [ 9600/60000]
loss: 1.146796  [12800/60000]
loss: 1.251547  [16000/60000]
loss: 1.241614  [19200/60000]
loss: 0.853472  [22400/60000]
loss: 1.063918  [25600/60000]
loss: 0.915745  [28800/60000